In [29]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import jqdatasdk
from jqdatasdk import *
# jqdatasdk.auth('15357927161', 'Yg15156688441')

## 获取实时数据

In [24]:
data = get_price('IF1605.CCFX', start_date='2023-05-01', end_date='2023-05-31', frequency='minute', fields=['open', 'close', 'volume'])
data

,open,close,volume
2023-05-04 09:31:00,NaN,NaN,NaN
2023-05-04 09:32:00,NaN,NaN,NaN
2023-05-04 09:33:00,NaN,NaN,NaN
2023-05-04 09:34:00,NaN,NaN,NaN
2023-05-04 09:35:00,NaN,NaN,NaN
...,...,...,...
2023-05-30 14:56:00,NaN,NaN,NaN
2023-05-30 14:57:00,NaN,NaN,NaN
2023-05-30 14:58:00,NaN,NaN,NaN
2023-05-30 14:59:00,NaN,NaN,NaN


## 模拟一个期货的价格用于回测

In [28]:
# 设置日期范围
start_date = '2023-05-01'
end_date = '2023-05-30'

# 创建日期时间索引
date_range = pd.date_range(start=start_date, end=end_date, freq='B')  # 'B' 表示工作日

# 每天的时间段
times = pd.date_range(start='9:30', end='15:15', freq='min').time

# 使用列表推导式生成每天每分钟的时间戳
timestamps = [pd.Timestamp(f'{date.date()} {time}') for date in date_range for time in times]

# 创建DataFrame
df = pd.DataFrame(index=timestamps, columns=['open'])

# 模拟价格数据
np.random.seed(24)
amplitude = 20  # 波动幅度
period = len(df) / 30  # 假设30天内有一个完整的周期
noise = np.random.normal(0, 2, size=len(df))  # 随机噪声

df['open'] = 1000 + amplitude * np.sin(np.linspace(0, 2 * np.pi, len(df))) + noise

# 输出数据查看
print(df.head())

# 将数据保存到CSV文件（可选）
df.to_csv('./data/raw/Futures_Price_Simulation.csv')

                            open
2023-05-01 09:30:00  1002.658424
2023-05-01 09:31:00   998.476444
2023-05-01 09:32:00   999.400461
2023-05-01 09:33:00   998.067912
2023-05-01 09:34:00   997.924411


## 拟合策略

In [44]:
def fit_and_evaluate(df):
    # convert time with the minute as unit
    df['minutes'] = np.arange(len(df))
    
    # Second poly fitting curve
    coefficients = np.polyfit(df['minutes'], df['price'], 2)
    poly = np.poly1d(coefficients)

    # Calculate first order derivative and second order derivative value
    derivative1 = poly.deriv(1)
    derivative2 = poly.deriv(2)
    last_minute = df['minutes'].iloc[-1]
    trend = derivative1(last_minute)
    acceleration = derivative2(last_minute)
    
    # return product for signal generation
    return trend, acceleration

def backtest_strategy(df, start_date, end_date, window_start_time, window_end_time, end_limit_time, initial_capital):
    window_start = datetime.strptime(window_start_time, '%H:%M')
    window_end_initial = datetime.strptime(window_end_time, '%H:%M')
    end_limit = datetime.strptime(end_limit_time, '%H:%M')

    capital = initial_capital
    shares_owned = 0
    position_open_price = 0

    results = []

    for single_date in pd.date_range(start_date, end_date):
        day_str = single_date.strftime('%Y-%m-%d')
        if day_str in df.index:
            filtered_data = df.loc[day_str]
            window_end = window_end_initial
            initial_window = filtered_data.between_time(window_start.strftime('%H:%M'), window_end.strftime('%H:%M'))
            initial_trend, initial_acceleration = fit_and_evaluate(initial_window)

            if initial_trend * initial_acceleration > 0:
                position_type = "Long" if initial_trend > 0 else "Short"
                # Assume opening price is the last price in the initial window
                position_open_price = initial_window.iloc[-1]
                # Calculate number of shares to buy (floor division to avoid fractional shares if applicable)
                shares_owned = capital // position_open_price
                capital -= shares_owned * position_open_price
                results.append(f"{day_str} - {position_type} position opened at {position_open_price}, shares owned: {shares_owned}")

                while window_end < end_limit:
                    window_end += timedelta(minutes=1)
                    window_end_str = window_end.strftime('%H:%M')
                    current_window = filtered_data.between_time(window_start.strftime('%H:%M'), window_end_str)
                    current_trend, current_signal = fit_and_evaluate(current_window)

                    if current_trend * initial_trend < 0 or current_signal * initial_acceleration < 0:
                        # Closing position at current last price
                        closing_price = current_window.iloc[-1]
                        capital += shares_owned * closing_price
                        results.append(f'{day_str} - Close Position at {window_end_str}, closing price: {closing_price}, capital: {capital}')
                        shares_owned = 0
                        break
                else:
                    # Forced close at end limit
                    closing_price = filtered_data.at[end_limit.strftime('%Y-%m-%d %H:%M'), 'Price']
                    capital += shares_owned * closing_price
                    results.append(f"{day_str} - Forced close position at {end_limit.strftime('%H:%M')}, closing price: {closing_price}, capital: {capital}")
                    shares_owned = 0
            else:
                results.append(f"{day_str} - Do not build position")

    return results, capital

,open
2023-05-01 09:30:00,1002.658424
2023-05-01 09:31:00,998.476444
2023-05-01 09:32:00,999.400461
2023-05-01 09:33:00,998.067912
2023-05-01 09:34:00,997.924411
...,...
2023-05-01 10:46:00,1002.623418
2023-05-01 10:47:00,1001.329400
2023-05-01 10:48:00,999.582003
2023-05-01 10:49:00,1005.266263


In [39]:
data = pd.read_csv('./data/raw/Futures_Price_Simulation.csv', index_col=0)
data.index = pd.to_datetime(data.index)